In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'n2o': 3.2e-07, 'ch4': 1.8e-06}
band = [9]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0]
  ng_refs = [7]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)]
  wgt = [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
 

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.141371                    1 -0.074758
0.000750        2 -0.140434                    2 -0.074114
0.001052        3 -0.138290                    3 -0.072747
0.001476        4 -0.136001                    4 -0.071255
0.002070        5 -0.133706                    5 -0.069788
0.002904        6 -0.131450                    6 -0.068242
0.004074        7 -0.129271                    7 -0.066730
0.005714        8 -0.127166                    8 -0.065137
0.008015        9 -0.125202                    9 -0.063591
0.011243       10 -0.123441                   10 -0.061965
0.015771       11 -0.121981                   11 -0.060353
0.022122       12 -0.120947                   12 -0.058779
0.031031       13 -0.120529                   13 -0.057195
0.043528       14 -0.120823                   14 -0.055587
0.061057       15 -0.116083                   15 -0.050062
0.085645       16 -0.094354                   16 -0.032873
0.120136       17 -0.055043                   17 -0.003231
0.168516       18 -0.005931                   18  0.034708
0.236378       19  0.053692                   19  0.081784
0.331549       20  0.125704                   20  0.139754
0.465100       21  0.212549                   21  0.210738
0.652400       22  0.317729                   22  0.296306
0.915100       23  0.412057                   23  0.374566
1.283650       24  0.398691                   24  0.376387
1.800600       25  0.292516                   25  0.304906
2.525700       26  0.191833                   26  0.224170
3.542800       27  0.119250                   27  0.153304
4.969550       28  0.072816                   28  0.097040
6.970850       29  0.046833                   29  0.057644
9.778100       30  0.032795                   30  0.033092
13.715850      31  0.023264                   31  0.018623
19.239350      32  0.015309                   32  0.010283
26.987250      33  0.010156                   33  0.006378
37.855300      34  0.006725                   34  0.003556
53.100050      35  0.001608                   35 -0.002013
73.887500      36 -0.005457                   36 -0.010353
97.662500      37 -0.011460                   37 -0.017900
121.437500     38 -0.016365                   38 -0.024269
145.212500     39 -0.023236                   39 -0.031823
168.987500     40 -0.030591                   40 -0.038930
192.762500     41 -0.032804                   41 -0.040950
216.537500     42 -0.028546                   42 -0.037042
240.312500     43 -0.022239                   43 -0.030710
264.087500     44 -0.014978                   44 -0.023099
287.862500     45 -0.007057                   45 -0.013944
311.637500     46 -0.000299                   46 -0.007041
335.412500     47  0.007086                   47 -0.000095
359.187500     48  0.015723                   48  0.008687
382.962500     49  0.025324                   49  0.019128
406.737500     50  0.035620                   50  0.030028
430.512500     51  0.046190                   51  0.040464
454.287500     52  0.055882                   52  0.049403
478.062500     53  0.065169                   53  0.058657
501.837500     54  0.074387                   54  0.068599
525.612500     55  0.083492                   55  0.079472
549.387500     56  0.092536                   56  0.090679
573.162500     57  0.101765                   57  0.101780
596.937500     58  0.114155                   58  0.115385
620.712500     59  0.124671                   59  0.126366
644.487500     60  0.132658                   60  0.134320
668.262500     61  0.138577                   61  0.140418
692.037500     62  0.142784                   62  0.145157
715.812500     63  0.145846                   63  0.151048
739.587500     64  0.148319                   64  0.157227
763.362500     65  0.150768                   65  0.162968
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -12.500557   0.000000 -12.500557 -12.517396  8.921780e-08   
0.000624    2     -12.500571   0.000004 -12.500567 -12.517404  2.442764e-06   
0.000876    3     -12.500577   0.000005 -12.500572 -12.517407  3.402029e-06   
0.001229    4     -12.500585   0.000007 -12.500577 -12.517412  4.777025e-06   
0.001723    5     -12.500596   0.000010 -12.500585 -12.517418  6.750801e-06   
0.002417    6     -12.500611   0.000015 -12.500596 -12.517426  9.581331e-06   
0.003391    7     -12.500633   0.000021 -12.500611 -12.517438  1.364393e-05   
0.004757    8     -12.500663   0.000030 -12.500632 -12.517455  1.947007e-05   
0.006672    9     -12.500705   0.000044 -12.500661 -12.517478  2.783080e-05   
0.009359    10    -12.500763   0.000062 -12.500701 -12.517510  3.981698e-05   
0.013128    11    -12.500845   0.000089 -12.500756 -12.517555  5.701327e-05   
0.018415    12    -12.500960   0.000127 -12.500833 -12.517618  8.167208e-05   
0.025830    13    -12.501120   0.000181 -12.500939 -12.517705  1.170064e-04   
0.036232    14    -12.501345   0.000258 -12.501087 -12.517826  1.676397e-04   
0.050823    15    -12.501662   0.000366 -12.501296 -12.517994  2.402235e-04   
0.071291    16    -12.502104   0.000526 -12.501578 -12.518225  3.491740e-04   
0.100000    17    -12.502691   0.000793 -12.501898 -12.518520  5.323725e-04   
0.140271    18    -12.503428   0.001267 -12.502161 -12.518865  8.619661e-04   
0.196760    19    -12.504304   0.002103 -12.502201 -12.519224  1.453289e-03   
0.275997    20    -12.505253   0.003556 -12.501697 -12.519507  2.504280e-03   
0.387100    21    -12.506087   0.006045 -12.500042 -12.519517  4.353536e-03   
0.543100    22    -12.506361   0.010246 -12.496115 -12.518848  7.579636e-03   
0.761700    23    -12.505092   0.017205 -12.487887 -12.516719  1.312434e-02   
1.068500    24    -12.500836   0.027924 -12.472912 -12.512083  2.210196e-02   
1.498800    25    -12.493714   0.041124 -12.452590 -12.505033  3.423950e-02   
2.102400    26    -12.485476   0.053801 -12.431675 -12.496826  4.783532e-02   
2.949000    27    -12.476915   0.064479 -12.412437 -12.488085  6.157802e-02   
4.136600    28    -12.468386   0.072726 -12.395661 -12.479159  7.422119e-02   
5.802500    29    -12.460157   0.078866 -12.381291 -12.470351  8.456438e-02   
8.139200    30    -12.452300   0.083973 -12.368328 -12.461847  9.201775e-02   
11.417000   31    -12.444909   0.089314 -12.355594 -12.453933  9.695424e-02   
16.014700   32    -12.438722   0.095798 -12.342924 -12.447625  1.007892e-01   
22.464000   33    -12.435496   0.104268 -12.331228 -12.444967  1.059878e-01   
31.510500   34    -12.437007   0.116663 -12.320344 -12.448440  1.162966e-01   
44.200100   35    -12.445435   0.135199 -12.310235 -12.461795  1.349976e-01   
62.000000   36    -12.467319   0.160473 -12.306846 -12.494116  1.630739e-01   
85.775000   37    -12.514159   0.191945 -12.322213 -12.559672  1.994697e-01   
109.550000  38    -12.578161   0.223672 -12.354489 -12.646591  2.359718e-01   
133.325000  39    -12.657891   0.257311 -12.400579 -12.752822  2.738469e-01   
157.100000  40    -12.757181   0.291161 -12.466020 -12.880402  3.117936e-01   
180.875000  41    -12.879875   0.327701 -12.552175 -13.029901  3.516407e-01   
204.650000  42    -13.022196   0.377636 -12.644560 -13.195787  4.021865e-01   
228.425000  43    -13.179000   0.454047 -12.724954 -13.373534  4.756023e-01   
252.200000  44    -13.350394   0.562809 -12.787585 -13.563188  5.787562e-01   
275.975000  45    -13.539543   0.709775 -12.829768 -13.767645  7.181515e-01   
299.750000  46    -13.751130   0.901486 -12.849644 -13.991331  9.025622e-01   
323.525000  47    -13.980399   1.129912 -12.850487 -14.229831  1.121231e+00   
347.300000  48    -14.224744   1.394212 -12.830532 -14.480600  1.371733e+00

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')